# ライブラリの準備

Pytorchのライブラリ(lib)をzip形式でGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5<br>
取得したリンクからidを抜き出す。<br>
**1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5**

In [0]:
# Google Cloud SDKで、Google Driveのファイルを、
# Google Colabolatoryの一時領域にダウンロードするための認証
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Pytorchのライブラリをダウンロード
id = '1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
import os
os.chdir('/content')
downloaded.GetContentFile('lib.zip')

In [0]:
# ダウンロードを確認
!pwd
%ls

In [0]:
# ライブラリを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "lib"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall() # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
os.remove(save_path) # ZIPファイルを消去

In [0]:
# 処理結果を確認
!pwd
!ls
%ls ./lib/

# データセットの準備

## データの格納／展開

データセット(zip)をローカルにダウンロードする。<br>
https://www.kaggle.com/daggysheep/find-a-car-park

ダウンロードしたデータセット(zip)をGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV<br>
取得したリンクからidを抜き出す。<br>
**17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV**

In [0]:
# データセットをダウンロード
%%time
id = '17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
os.chdir('/content')
downloaded.GetContentFile('find-a-car-park.zip')

In [0]:
# ダウンロードを確認
!pwd
%ls

In [0]:
# データセットを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "find-a-car-park"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall(save_name) # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
#os.remove(save_path) # ZIPファイルを消去 # 試験中は消さない方が良い

In [0]:
# 処理結果を確認
%ls ./find-a-car-park/data

In [0]:
# データ個数を確認
!ls -1U "/content/find-a-car-park/data/Free" | wc -l
!ls -1U "/content/find-a-car-park/data/Full" | wc -l

## trainデータ／validationデータの分割

In [0]:
os.chdir('/content/find-a-car-park')
!pwd
%ls

In [0]:
# trainデータ、valデータを格納するフォルダを作成
os.mkdir("/content/find-a-car-park/train/")
os.mkdir("/content/find-a-car-park/val/")
!pwd
%ls

In [0]:
# フォルダが空であることを確認
!ls "/content/find-a-car-park/train/"
!ls -1U "/content/find-a-car-park/train/" | wc -l
!ls "/content/find-a-car-park/val/"
!ls -1U "/content/find-a-car-park/val/" | wc -l

In [0]:
#「/content/find-a-car-park/data/」以下の画像を「train」「val」フォルダに振り分ける
# train:val = 3:1
import glob
import shutil
os.chdir('/content/find-a-car-park/data')
label_dirs = os.listdir(path='.')
path = os.getcwd()  

# ラベル名が付与されている各フォルダごとに、中身のファイル名を取り出す
for dir_name in label_dirs:
  files = os.listdir(path + "/" + dir_name) # 1個のラベルフォルダのパスを取得
  numof_data = len(files) # ラベルフォルダ中のデータ数を取得
  files = glob.glob("./" + dir_name + "/*") # ラベルフォルダ中の全ファイルリストを取得

  # ラベルフォルダ中のファイルを抽出して、"val"フォルダに移動する
  # 全体の1/4のデータを"val"フォルダに移動するまで繰り返す
  i = 0
  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    i = i + 1
    if i < numof_data * 1/4:
      data_dir = "/content/find-a-car-park/val/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
    else:
      data_dir = "/content/find-a-car-park/train/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
  shutil.rmtree("./" + dir_name) # データを振り分け終えたラベルフォルダは削除しておく

In [0]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/train/Free" | wc -l
!ls -1U "/content/find-a-car-park/train/Full" | wc -l

In [0]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/val/Free" | wc -l
!ls -1U "/content/find-a-car-park/val/Full" | wc -l

In [0]:
# [Option] データセットを一旦消す場合に実行
#%rm -r '/content/find-a-car-park'

## [DELETED] Google Drive上でデータセットを解凍して使用する場合

In [0]:
# Google Driveをmountする。
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park.zip'

In [0]:
!ls

In [0]:
!mkdir '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

In [0]:
!mv '/content/data' '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

# 学習／検証

In [0]:
%%time
os.chdir('/content/')
!npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

In [0]:
%%time
import time
time.sleep(5)
! cat url.txt

In [0]:
!pwd
!ls

In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [0]:
# アーキテクチャを選択して使用する。
%%time
net = network.VGG16(class_size=2)
#net = network.InceptionV3(class_size=2)
#net = network.ResNet50(class_size=2)

In [0]:
%%time
batch_size = 64
# フォルダのパスはかえる
train_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/train/', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/val/', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

In [0]:
%%time
model = architecture.CNN_Architecture(net)

In [0]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

In [0]:
%%time
model.predict(data_loader['val'])

------------------